https://www.crummy.com/software/BeautifulSoup/bs4/doc/
    

* white list domains, always interesting
* curate the words in both lists? maybe that'll clean up the final score.  or just get more sample data in of things you like.


In [1]:
#### from bs4 import BeautifulSoup
import requests
import functools
import re
import operator
from bs4 import BeautifulSoup

ignore_list = ['about','include','other','import','update','download']

def get_words_site(url):
    try:
        soup = BeautifulSoup( requests.get(url).content.decode('utf-8') )
        l1 = filter(lambda x: re.match(r'^[a-z].*', x),   set(soup.get_text().lower().split()) )
        l2 = map( lambda x: re.sub("[^a-zA-Z]+", "", x), l1)
        l3 = filter(lambda x: len(x) >4, l2)
        l4 = filter(lambda x: x not in ignore_list, l3)
    except:
        return False
    
    return list(l4)


In [94]:
# Load data into database

import sqlite3

con = sqlite3.connect("data_collect/links.db")
cur = con.cursor()

"""
insert = []
for l in open("data_collect/import.txt"):
    (url, state) = l.split(",")
    state = int(state)
    insert.append( (url, state, 0) )
    
cur.executemany(
    "REPLACE into links (url, rating, scanned) values (?,?,?)", insert 
)
con.commit()
con.close()
"""

# ---

# Download words into the database
con = sqlite3.connect("data_collect/links.db")
cur = con.cursor()

for row in cur.execute("select (url) from links where scanned=0").fetchall():
    url = row[0]
    words = get_words_site( url)
    if words:
        print(url)
        cur.execute("update links set words=? where url=?", (",".join(words), url ) )
        cur.execute("update links set scanned=? where url=?", (True, url) )
        con.commit()
    else:
        print("Failed to load {}".format(url))

con.close()

# ---

# Load words
con = sqlite3.connect("data_collect/links.db")
cur = con.cursor()

word_list = {}
for row in cur.execute("select * from links where scanned=1").fetchall():
    (url, rating, words, scanned) = row
    for word in words.split(','):
        # [spam_count, ham_count]
        word_list.setdefault(word, [0,0])
        if rating == 0:
            # spam
            word_list[word][0] += 1
        else:
            # ham
            word_list[word][1] += 1

# word_list = dict(filter(lambda e: e[1][0] > 20 and e[1][1] == 0 , word_list.items()))

spam_url_count = cur.execute("select count(url) from links where rating=0").fetchall()[0][0]
ham_url_count  = cur.execute("select count(url) from links where rating=1").fetchall()[0][0]
        
con.close()

ham_words = list(filter( lambda x: word_list[x][1] > 0, word_list.keys() ))
spam_words = list(filter( lambda x: word_list[x][0] > 0, word_list.keys() ))

only_ham_words = set(ham_words) - set(spam_words)
only_spam_words = set(spam_words) - set(ham_words)
all_words = set(ham_words) | set(spam_words)

# ---

#
# TODO: instead of all_words, build a list of custom words to check for 
#
scores = {}
for word in all_words:
    spam_scores = (word_list[word][0]+1) / (spam_url_count+2)
    ham_scores = (word_list[word][1]+1) / (ham_url_count+2)
    scores[word] = [spam_scores, ham_scores]
    

https://www.latimes.com/homeless-housing/story/2022-06-20/california-affordable-housing-cost-1-million-apartment
https://jacobhrussell.com/blog/mighty
https://www.practicalmachinist.com/inspection-and-measurement/top-5-digital-calipers-for-machinists/
https://engineering.fb.com/2022/06/20/data-infrastructure/transparent-memory-offloading-more-memory-at-a-fraction-of-the-cost-and-power/
https://publicdomainreview.org/collection/shores-of-the-polar-sea/
https://tuple.app/blog/sso-should-be-table-stakes
https://github.com/sonnyt/just
https://dantepfer.com/blog/?p=277
https://blog.efitz.net/blog/icon-activism/
https://eli.thegreenplace.net/2022/why-is-it-easy-to-implement-a-lisp/
https://www.irishtimes.com/culture/tv-radio/2022/06/18/rowan-atkinson-in-a-proper-free-society-you-should-be-allowed-to-make-jokes-about-absolutely-anything/
https://fasterthanli.me/articles/remote-development-with-rust-on-fly-io
https://twitter.com/culturaltutor/status/1538211892707086338
https://oa.mg/blog/watch

In [106]:
## Calculate if a site is spam or not
#site_words = get_words_site("https://tylercipriani.com/blog/2022/04/30/ive-used-all-the-notebooks/")
site_words = get_words_site("https://www.theregister.com/2022/05/19/intel_immersion_lab/")
#site_words = get_words_site("https://kk.org/thetechnium/bicycle-haiku/")
#site_words = get_words_site("https://justine.lol/redbean2/")
#site_words = ['blinkenlightsgooglegroupscom']

probs = []
p2 = []

pr_s = spam_url_count / ( spam_url_count + ham_url_count)
pr_h = ham_url_count / ( spam_url_count + ham_url_count)

# statically setting this number helps get to a saner result
pr_s = .5
pr_h = .5

spam_total=0
ham_total=0
for word in site_words:
    pr_ws = scores.get( word, [.5, 0])[0]
    pr_wh = scores.get( word, [0, .5])[1] 
    spam_total += pr_ws
    ham_total += pr_wh
    
final = (pr_s * spam_total) / ( (pr_s * spam_total) + (pr_h * ham_total) )

print(final)
if final > .5:
    print("Spam", final)
else:
    print("ok", final)
    


0.5251506579756487
Spam 0.5251506579756487


In [59]:
word_list['blinkenlightsgooglegroupscom'] / 

[0, 1]

In [61]:
scores['blinkenlightsgooglegroupscom']

[0.0004111842105263158, 0.023305084745762712]